<hr>

<font color=skyblue>**Data preparations**</font>

以下提供兩種模糊影像的處理方式，以做為訓練與驗證資料。分別為：
- Data Processing 1: 將清晰之影像切成小圖（patch），再逐一進行高斯模糊化，並分別儲存清晰與模糊影像。即以小圖做為訓練資料。
- Data Processing 2: 直接將原圖進行高斯模糊化，並儲存模糊影像。即以大圖做為訓練資料。

<hr>

<font color=skyblue>套件宣告</font>

In [18]:
from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
import patchify
import numpy as np
import matplotlib.gridspec as gridspec
import glob as glob
import os
import cv2
import shutil

<hr>

<font color=skyblue>**Data Preprocessing 1**: Dataset Patch Creation and blurred patch creation</font>

This script is used to create patches of size (33, 33, 3) from the images in the T91 dataset.
The patches are saved in the following directories:
- inputs/sharp_patches
- inputs/blurred_patches

<hr>

<font color=skyblue>常數宣告</font>

In [36]:
DIR_PATH = '../'
SHOW_PATCHES = False
STRIDE = 14 # Stride for the patches.
SIZE = 35 # Size of the patches.
GAUSSIAN_KERNEL_SIZE_PATCH = (7, 7) # Kernel size for Gaussian blur.
SIGMAX = 0 # Sigma for Gaussian blur. 0 means it is calculated from the kernel size.


<font color=skyblue>副程式宣告</font>

In [33]:
def show_patches(patches):
    """
    Function to plot the patches.
    """
    plt.figure(figsize=(patches.shape[0], patches.shape[1]))
    gs = gridspec.GridSpec(patches.shape[0], patches.shape[1])
    gs.update(wspace=0.01, hspace=0.02)
    counter = 0
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            ax = plt.subplot(gs[counter])
            plt.imshow(patches[i, j, 0, :, :, :])
            plt.axis('off')
            counter += 1
    plt.show()

def create_patches(input_paths, out_sharp_path, out_blur_path):
    '''
    Function to create patches from the images in the input paths.
    The patches are saved in the output paths.'''
    
    os.makedirs(out_sharp_path, exist_ok=True)
    os.makedirs(out_blur_path, exist_ok=True)
    all_paths = []

    for input_path in input_paths:
        all_paths.extend(glob.glob(f"{input_path}/*"))
    print(f"Creating patches for {len(all_paths)} images")

    for image_path in tqdm(all_paths, total=len(all_paths)):
        image = Image.open(image_path)
        image_name = image_path.split(os.path.sep)[-1].split('.')[0]
        
        patches = patchify.patchify(np.array(image), (SIZE, SIZE, 3), STRIDE)
        if SHOW_PATCHES:
            show_patches(patches)

        # print(patches.shape)
        counter = 0
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                counter += 1
                patch = patches[i, j, 0, :, :, :]
                patch = cv2.cvtColor(patch, cv2.COLOR_RGB2BGR)
                cv2.imwrite(
                    f"{out_sharp_path}/{image_name}_{counter}.png",
                    patch
                )
                blur = cv2.GaussianBlur(patch, GAUSSIAN_KERNEL_SIZE_PATCH, sigmaX=SIGMAX)

                cv2.imwrite(
                    f"{out_blur_path}/{image_name}_{counter}.png",
                    blur
                )


自己寫的，有修改內容

In [37]:
def show_patches(patches):
    """
    Function to plot the patches.
    """
    plt.figure(figsize=(patches.shape[0], patches.shape[1]))
    gs = gridspec.GridSpec(patches.shape[0], patches.shape[1])
    gs.update(wspace=0.01, hspace=0.02)
    counter = 0
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            ax = plt.subplot(gs[counter])
            plt.imshow(patches[i, j, 0, :, :, :])
            plt.axis('off')
            counter += 1
    plt.show()

def create_patches(input_paths, out_sharp_path, out_blur_path):
    '''
    Function to create patches from the images in the input paths.
    The patches are saved in the output paths.'''
    
    os.makedirs(out_sharp_path, exist_ok=True)
    os.makedirs(out_blur_path, exist_ok=True)
    all_paths = []

    for input_path in input_paths:
        all_paths.extend(glob.glob(f"{input_path}/*"))
    print(f"Creating patches for {len(all_paths)} images")

    for image_path in tqdm(all_paths, total=len(all_paths)):
        image = Image.open(image_path)
        image_name = image_path.split(os.path.sep)[-1].split('.')[0]
        
        patches = patchify.patchify(np.array(image), (SIZE, SIZE, 3), STRIDE)
        if SHOW_PATCHES:
            show_patches(patches)

        # print(patches.shape)
        counter = 0
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                counter += 1
                patch = patches[i, j, 0, :, :, :]
                patch = cv2.cvtColor(patch, cv2.COLOR_RGB2BGR)
                cv2.imwrite(
                    f"{out_sharp_path}/{image_name}_{counter}.png",
                    patch
                )
                # Step 1: apply Gaussian blur
                blur = cv2.GaussianBlur(patch, GAUSSIAN_KERNEL_SIZE_PATCH, sigmaX=SIGMAX)
                # Step 2: resize to 1/3 size
                height, width = blur.shape[:2]
                small = cv2.resize(blur, (width // 3, height // 3), interpolation=cv2.INTER_AREA)
                # Step 3: resize back to original size
                restored = cv2.resize(small, (width, height), interpolation=cv2.INTER_LINEAR)

                cv2.imwrite(
                    f"{out_blur_path}/{image_name}_{counter}.png",
                    restored
                )


<font color=skyblue>執行圖像裁切</font>

- 清晰小圖另存目錄，作為 labels
- 模糊小圖另存目錄，作為 inputs

In [38]:
input_paths =  [f"{DIR_PATH}inputs/T91/"]
outputs_sharp_path = DIR_PATH + "inputs/T91_sharp_patches"
outputs_blur_path = DIR_PATH + "inputs/T91_blurred_patches"

# Check and remove if folders exist
if os.path.exists(outputs_sharp_path):
    shutil.rmtree(outputs_sharp_path)
    print(f"Removed existing folder: {outputs_sharp_path}")

if os.path.exists(outputs_blur_path):
    shutil.rmtree(outputs_blur_path)
    print(f"Removed existing folder: {outputs_blur_path}")

# input_paths =  [f"{DIR_PATH}inputs/T30/"]
# outputs_sharp_path = DIR_PATH + "inputs/T30_sharp_patches"
# outputs_blur_path = DIR_PATH + "inputs/T30_blurred_patches"

create_patches(input_paths, outputs_sharp_path, outputs_blur_path)
print('DONE')

Removed existing folder: ../inputs/T91_sharp_patches
Removed existing folder: ../inputs/T91_blurred_patches
Creating patches for 91 images


100%|██████████| 91/91 [00:40<00:00,  2.27it/s]

DONE


自己修改的部分：

SIZE = 64 # Size of the patches. \
GAUSSIAN_KERNEL_SIZE_PATCH = (5, 5) # Kernel size for Gaussian blur. \
SIGMAX = 0 # Sigma for Gaussian blur. 0 means it is calculated from the kernel size.

這部分是老師說的縮小 1/3 再放大 1/3 的部分
\# Step 2: resize to 1/3 size \
height, width = blur.shape[:2] \
small = cv2.resize(blur, (width // 3, height // 3), interpolation=cv2.INTER_AREA) \
\# Step 3: resize back to original size \
restored = cv2.resize(small, (width, height), interpolation=cv2.INTER_LINEAR)

<hr>

<font color=skyblue>**Data Preprocessing 2**: Prepare blurred images by adding Gaussian blurring to the original (sharp) images</font>

- sharp images : 可以來自 T91, General100, Kaggle_sharp_350
- blurring images : gaussian_blurred，可以使用單一的模糊程度或混合不同模糊程度
- 保留原影像大小

<hr>


<font color=skyblue>常數宣告</font>

In [5]:
DIR_PATH = '../'
GAUSSIAN_KERNEL_SIZE_ORIGINAL = (31, 31)
SIGMAX = 0 # Sigma for Gaussian blur. 0 means it is calculated from the kernel size.

<font color=skyblue>執行圖像高斯模糊化</font>

- 保留輸入圖像目錄作為清晰圖（sharp, labels）
- 輸出目錄為模糊圖（blurred）
- 輸出圖像大小與輸入圖像大小相同

In [6]:
src_dir = DIR_PATH+'inputs/General100'
images = os.listdir(src_dir)
dst_dir = DIR_PATH+'inputs/gaussian_blurred'
os.makedirs(dst_dir, exist_ok=True)

for i, img in tqdm(enumerate(images), total=len(images)):
    img = cv2.imread(f"{src_dir}/{images[i]}", cv2.IMREAD_COLOR)
    # add gaussian blurring
    # blur = cv2.GaussianBlur(img, (5, 5), 0)  # Mild blur
    # blur = cv2.GaussianBlur(img, (15, 15), 5)  # Stronger blur

    blur = cv2.GaussianBlur(img, GAUSSIAN_KERNEL_SIZE_ORIGINAL, sigmaX=SIGMAX)
    cv2.imwrite(f"{dst_dir}/{images[i]}", blur)

print('DONE')

100%|██████████| 100/100 [00:01<00:00, 59.36it/s]

DONE
